https://github.com/combust/mleap-demo/blob/master/notebooks/PySpark%20-%20AirBnb.ipynb

https://combust.github.io/mleap-docs/py-spark/



In [3]:
!pip install mleap

     |████████████████████████████████| 59 kB 810 kB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 3.5 MB/s eta 0:00:01
  Created wheel for scikit-learn: filename=scikit_learn-0.22.2.post1-cp39-cp39-linux_x86_64.whl size=7253431 sha256=4ad4a08447eea900c82cb1161d8c5e8478b66928d2e9a93a993076da4c57d6ed
  Stored in directory: /home/jovyan/.cache/pip/wheels/f0/8b/2a/7e1e1a081fbbd083335a795448aca004ad4736ef1d7d4ae062
Successfully built scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


In [9]:
# download file
!wget -O airbnb.csv https://github.com/combust/mleap-demo/blob/master/data/airbnb.csv?raw=true

--2021-09-16 21:18:54--  https://github.com/combust/mleap-demo/blob/master/data/airbnb.csv?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/combust/mleap-demo/raw/master/data/airbnb.csv [following]
--2021-09-16 21:18:54--  https://github.com/combust/mleap-demo/raw/master/data/airbnb.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/combust/mleap-demo/master/data/airbnb.csv [following]
--2021-09-16 21:18:54--  https://raw.githubusercontent.com/combust/mleap-demo/master/data/airbnb.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting 

In [10]:
!head airbnb.csv

id,name,price,bedrooms,bathrooms,room_type,square_feet,host_is_superhost,state,cancellation_policy,security_deposit,cleaning_fee,extra_people,number_of_reviews,price_per_bedroom,review_scores_rating,instant_bookable
1949687,Delectable Victorian Flat for two,80.0,1.0,1.0,Entire home/apt,,0.0,London,moderate,100.0,20.0,10.0,8,80.0,94.0,0.0
3863509,Fully Furnished 3 Bed House/Garden,40.0,1.0,1.0,Private room,,0.0,London,flexible,0.0,0.0,0.0,5,40.0,55.0,0.0
1988980,Cozy Double Room in Victorian House,35.0,1.0,1.0,Private room,,0.0,Greenwich,flexible,0.0,5.0,10.0,32,35.0,89.0,0.0
2347198,Double Room In Central London.,42.0,1.0,1.0,Private room,,0.0,London,strict,0.0,0.0,15.0,24,42.0,71.0,0.0
144337,Fast WIFI Breakfast FREE Parking 5,200.0,1.0,1.5,Private room,250.0,0.0,London,strict,300.0,0.0,20.0,24,200.0,84.0,0.0
1372647,Double Room Queens Park/Maida Vale,75.0,1.0,1.0,Private room,,0.0,London,flexible,0.0,0.0,10.0,3,75.0,100.0,0.0
2440394,Penthouse bedroom /Private Bath,70.0,1.0,1.0,Priva

In [13]:
from mleap import pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression

In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]") \
                    .appName('mleap') \
                    .getOrCreate()

In [19]:
df = spark.read.option("header", True).csv("airbnb.csv")

In [20]:
datasetFiltered = df.filter("price >= 50 AND price <= 750 and bathrooms > 0.0")
print(df.count())
print(datasetFiltered.count())

389255
321588


In [21]:
datasetFiltered.registerTempTable("df")

In [23]:
spark.sql("""
    select 
        state,
        count(*) as n,
        cast(avg(price) as decimal(12,2)) as avg_price,
        max(price) as max_price
    from df
    group by state
    order by count(*) desc
""").show()

+-------------+-----+---------+---------+
|        state|    n|avg_price|max_price|
+-------------+-----+---------+---------+
|           NY|48362|   146.75|     99.0|
|           CA|44716|   158.76|     99.0|
|Île-de-France|40732|   107.74|     99.0|
|       London|17542|   117.72|     99.0|
|          NSW|14416|   167.96|     99.0|
|       Berlin|13098|    81.01|     99.0|
|Noord-Holland| 8890|   128.56|     99.0|
|          VIC| 8636|   144.49|     99.0|
|North Holland| 7636|   134.60|     99.0|
|           IL| 7544|   141.85|     99.0|
|           ON| 7186|   129.05|     99.0|
|           TX| 6702|   196.59|     99.0|
|           WA| 5858|   132.48|     99.0|
|    Catalonia| 5748|   106.39|     99.0|
|           BC| 5522|   133.14|     99.0|
|           DC| 5476|   136.56|     99.0|
|       Québec| 5116|   104.98|     98.0|
|    Catalunya| 4570|    99.36|     99.0|
|       Veneto| 4486|   131.71|     99.0|
|           OR| 4330|   114.02|     99.0|
+-------------+-----+---------+---